In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import numpy as np
from batchify import Corpus
import random
import torch.optim as optim
from torch.autograd import Variable
from model import Encoder, Decoder
from math import log
from numpy import array
from numpy import argmax

In [2]:
corpus=Corpus()

In [3]:
device=torch.device('cuda')
print (device)

cuda


In [4]:
def beam_search_decoder(data, k):
    sequences = [[list(), 1.0]]
    # walk over each step in sequence
    for row in data:
        all_candidates = list()
        # expand each current candidate
        for i in range(len(sequences)):
            seq, score = sequences[i]
            for j in range(len(row)):
                candidate = [seq + [j], score * -1*(row[j])]
                all_candidates.append(candidate)
        # order all candidates by score
        ordered = sorted(all_candidates, key=lambda tup:tup[1])
        # select k best
        sequences = ordered[:k]
    return sequences
                
        

In [5]:
def validation(input_tensor, target_tensor, encoder, decoder, criterion,corpus):
    with torch.no_grad():
        encoded, hidden=encoder(input_tensor.long())
        batch_size=target_tensor.size(0)
        decoder_input = np.zeros((batch_size,1))
        decoder_input[:]=corpus.w2i['<sos>']
        decoder_input=torch.from_numpy(decoder_input).cuda().detach()
        dec_len=target_tensor.size(1)
        decoded, hidden,outputs=decoder(encoded,hidden,decoder_input.long(),dec_len,target_tensor,val=True)
    return outputs
        #s=outputs.size(1)
        #loss=0
        #for i in range(s):
        #    loss+=criterion(outputs[:,i,:],target_tensor[:,i].long())
        
        
    #return loss/target_tensor.size(1)

In [6]:
encoder=Encoder(vocab_size=50000, embedding_size=128, hidden_size=256)
decoder=Decoder(vocab_size=50000, embedding_dim=128, hidden_dim=256)
encoder=encoder.to(device)
decoder=decoder.to(device)

In [7]:
encoder_optimizer = optim.Adam(encoder.parameters())
decoder_optimizer = optim.Adam(decoder.parameters())
criterion=nn.NLLLoss()

In [25]:
checkpoint = torch.load('model.pth')
encoder.load_state_dict(checkpoint['encoder_state_dict'])
decoder.load_state_dict(checkpoint['decoder_state_dict'])
encoder_optimizer.load_state_dict(checkpoint['encoder_optimizer_state_dict'])
decoder_optimizer.load_state_dict(checkpoint['decoder_optimizer_state_dict'])
step = checkpoint['step']
tl = checkpoint['training_loss']
vl = checkpoint['validation_loss']

encoder.train()
decoder.train()

Decoder(
  (attn): Linear(in_features=384, out_features=400, bias=True)
  (attn_combine): Linear(in_features=384, out_features=128, bias=True)
  (linear1): Linear(in_features=512, out_features=256, bias=True)
  (linear2): Linear(in_features=512, out_features=256, bias=True)
  (embedding): Embedding(50000, 128)
  (gru): GRU(128, 256, batch_first=True)
  (linear): Linear(in_features=256, out_features=50000, bias=True)
)

In [ ]:
input_tensor_val,target_tensor_val=corpus.get_validation_batch()
input_tensor_val=Variable(input_tensor_val.cuda())
target_tensor_val=Variable(target_tensor_val.cuda())

outputs=validation(input_tensor_val[16].unsqueeze(0),target_tensor_val[16].unsqueeze(0), encoder, decoder, criterion,corpus)
outputs=outputs.squeeze(0)
seq = beam_search_decoder(outputs.cpu().tolist(), 3)

for i in seq:
    sentence=[corpus.i2w[x] for x in i[0]]
    print(" ".join(sentence))
    print('\n')